<a href="https://colab.research.google.com/github/AdilsonFilho23/Trabalho-Big-data/blob/main/trabalho_big_data_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
# Visualizando a base de treino
treino = pd.read_csv('train.csv')
treino.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
# Visualizando a base de teste
teste = pd.read_csv('test.csv')
teste.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [4]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [5]:
# Substituir valores nulos na coluna de idade por uma média
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.mean()

In [6]:
# Substituir a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

In [7]:
# Verificando as colunas de texto na base de treino
treino.columns[treino.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

In [8]:
# Verificando os valores na coluna Sex
treino.Sex.value_counts()

,count
Sex,
male,577
female,314


In [9]:
# e na coluna Embarked
treino.Embarked.value_counts()

,count
Embarked,
S,644
C,168
Q,77


In [10]:
# Usando uma lambda function para fazer esse tratamento
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [11]:
# Verificando os valores
treino[['Sex','MaleCheck']].value_counts()

,,count
Sex,MaleCheck,
male,1,577
female,0,314


In [12]:
# Usando uma lambda function para fazer esse tratamento
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [13]:
# Verificando os valores
teste[['Sex','MaleCheck']].value_counts()

,,count
Sex,MaleCheck,
male,1,266
female,0,152


In [14]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')

In [15]:
# Fit com os dados
ohe = ohe.fit(treino[['Embarked']])

In [16]:
# Fazendo a transformação
ohe.transform(treino[['Embarked']]).toarray()

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       ...,
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]], dtype=int32)

In [17]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(treino[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0


In [18]:
# Adicionar a coluna na base de treino
treino = pd.concat([treino,ohe_df],axis=1)

In [19]:
# Verificando os valores
treino[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

,,,,count
Embarked,Embarked_C,Embarked_Q,Embarked_S,
S,0,0,1,644
C,1,0,0,168
Q,0,1,0,77


In [20]:
# Transformando o resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(teste[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())

In [21]:
# Adicionando o resultado na base de teste
teste = pd.concat([teste,ohe_df],axis=1)

In [22]:
# Verificando também os valores
teste[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

,,,,count
Embarked,Embarked_C,Embarked_Q,Embarked_S,
S,0,0,1,270
C,1,0,0,102
Q,0,1,0,46


In [23]:
# Visualizando a base
treino.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1,0
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1,0


In [24]:
# Apagar a colunas tratadas
treino = treino.drop(['Sex','Embarked'],axis=1)
teste = teste.drop(['Sex','Embarked'],axis=1)

In [25]:
# Separando a base de treino em X e y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

In [26]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
# Criando o classificador da árvore de classificação
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [28]:
# Fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

In [29]:
# Previsão
y_pred_ac = clf_ac.predict(X_val)

In [30]:
# Criando o classificador do K-Nearest Neighbors
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [31]:
# Fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

In [32]:
# Previsão
y_pred_knn = clf_knn.predict(X_val)

In [33]:
# Criando o classificador da regressão logística
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

In [34]:
# Fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

In [35]:
# Previsão
y_pred_rl = clf_rl.predict(X_val)

In [36]:
# Score para a árvore
accuracy_score(y_val, y_pred_ac)

0.7559322033898305

In [37]:
# Score para o knn
accuracy_score(y_val, y_pred_knn)

0.7152542372881356

In [38]:
# Score para a regressão logística
accuracy_score(y_val, y_pred_rl)

0.8169491525423729

In [39]:
# Matriz de confusão para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[138,  37],
       [ 35,  85]])

In [40]:
# Matriz de confusão para o knn
confusion_matrix(y_val, y_pred_knn)

array([[147,  28],
       [ 56,  64]])

In [41]:
# Matriz de confusão para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[153,  22],
       [ 32,  88]])

In [42]:
# Visualizando o X_train
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
6,1,54.000000,0,0,51.8625,1,0,0,1,0
718,3,29.699118,0,0,15.5000,1,0,1,0,0
685,2,25.000000,1,2,41.5792,1,1,0,0,0


In [43]:
teste.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,892,3,34.5,0,0,7.8292,1,0,1,0,0
1,893,3,47.0,1,0,7.0000,0,0,0,1,0
2,894,2,62.0,0,0,9.6875,1,0,1,0,0


In [44]:
# Eliminar a coluna de id para a base de teste ser igual a base de treino
X_teste = teste.drop('PassengerId',axis=1)

In [45]:
training_features = X_train.columns

# Garantir que estejam na mesma coluna que X_train
X_teste = X_teste[training_features]

# Utilizar a regressão logística no teste
y_pred_rl = clf_rl.predict(X_teste)

In [46]:
# Criando uma nova coluna com a previsão na base de teste
teste['Survived'] = y_pred_rl

In [47]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[['PassengerId','Survived']]

In [48]:
# Exportar para um csv
base_envio.to_csv('resultados4.csv',index=False)